<a href="https://colab.research.google.com/github/aljeshishe/colab/blob/main/dividends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/mbk-dev/okama/blob/master/examples/01%20howto.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

You can run the code examples in [Google Colab](https://colab.research.google.com/github/mbk-dev/okama/blob/master/examples/01%20howto.ipynb).

**To install the package:**

In [1]:
!pip install -U okama plotly

     |████████████████████████████████| 81 kB 3.8 MB/s 
     |████████████████████████████████| 26.5 MB 1.4 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [2]:
import okama as ok
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12.0, 6.0]
pd.options.plotting.backend = 'plotly'
pd.options.display.max_rows = None

In [10]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [4]:
from gspread_dataframe import set_with_dataframe, get_as_dataframe

name = 'dividends'
# name = 'test'
sheet = gc.open(name).sheet1
print(f'Url: https://docs.google.com/spreadsheets/d/{sheet.spreadsheet.id}/edit#gid=0')
new_df = get_as_dataframe(sheet)
tickers = new_df.dropna(how='all')[['symbol']]
tickers.head(5)

,symbol
0,LNZLP.MOEX
1,LNZL.MOEX
2,NLMK.MOEX
3,MTLRP.MOEX
4,CHMF.MOEX


In [5]:
def get_div(row):
  x = ok.Asset(row.symbol)
  div = x.dividends
  div.index = div.index.to_timestamp()

  div_value = div[:'2022-01'].last('12M').sum() if '2021' in div.index else float('nan')
  forecast_div_value = div.last('12M').sum() if '2022' in div.index else float('nan')

  row['ticker'] = row.symbol.split('.')[0]
  row['div_value'] = div_value
  row['forecast_div_value'] = forecast_div_value
  row['inc'] = div_value / x.price * 100
  row['forecast_inc'] = forecast_div_value / x.price * 100
  row['name'] = x.name
  print(row.to_dict())
  return row

new_data_df = tickers.apply(get_div, axis='columns', result_type='expand').round(2)
new_data_df.head(5)

{'symbol': 'LNZLP.MOEX', 'ticker': 'LNZLP', 'div_value': 3699.27, 'forecast_div_value': 0.0, 'inc': 218.24601769911504, 'forecast_inc': 0.0, 'name': 'Lena Gold-Mining Public Joint Stock Company Lenzoloto'}
{'symbol': 'LNZL.MOEX', 'ticker': 'LNZL', 'div_value': 15219.5, 'forecast_div_value': 0.0, 'inc': 159.8686974789916, 'forecast_inc': 0.0, 'name': 'Lena Gold-Mining Public Joint Stock Company Lenzoloto'}
{'symbol': 'NLMK.MOEX', 'ticker': 'NLMK', 'div_value': 41.91, 'forecast_div_value': 40.01, 'inc': 19.635494752623686, 'forecast_inc': 18.745314842578708, 'name': 'Public Joint Stock Company "Novolipetsk Steel"'}
{'symbol': 'MTLRP.MOEX', 'ticker': 'MTLRP', 'div_value': 1.17, 'forecast_div_value': 58.89, 'inc': 0.3795620437956204, 'forecast_inc': 19.104622871046228, 'name': 'Mechel PAO'}
{'symbol': 'CHMF.MOEX', 'ticker': 'CHMF', 'div_value': 253.42000000000002, 'forecast_div_value': 310.23, 'inc': 15.990661282180717, 'forecast_inc': 19.575340737001515, 'name': 'Public Joint Stock Compan

,symbol,ticker,div_value,forecast_div_value,inc,forecast_inc,name
0,LNZLP.MOEX,LNZLP,3699.27,0.00,218.25,0.00,Lena Gold-Mining Public Joint Stock Company Le...
1,LNZL.MOEX,LNZL,15219.50,0.00,159.87,0.00,Lena Gold-Mining Public Joint Stock Company Le...
2,NLMK.MOEX,NLMK,41.91,40.01,19.64,18.75,"Public Joint Stock Company ""Novolipetsk Steel"""
3,MTLRP.MOEX,MTLRP,1.17,58.89,0.38,19.10,Mechel PAO
4,CHMF.MOEX,CHMF,253.42,310.23,15.99,19.58,Public Joint Stock Company Severstal


In [6]:
def get_prices(row):
  x = ok.Asset(row.symbol)
  prices = x.close_daily
  prices.index = prices.index.to_timestamp()
  row['price'] = x.price
  row['30d_min'] = prices.last('30D').min()
  row['30d_max'] = prices.last('30D').max()
  row['30d_drowdown'] = x.price / row['30d_max'] * 100 - 100
  row['365d_min'] = prices.last('365D').min()
  row['365d_max'] = prices.last('365D').max()
  row['365d_drowdown'] = x.price / row['365d_max'] * 100 - 100
  print(row.to_dict())
  return row

new_data_df = new_data_df.apply(get_prices, axis='columns', result_type='expand').round(2)
new_data_df.head(5)


{'symbol': 'LNZLP.MOEX', 'ticker': 'LNZLP', 'div_value': 3699.27, 'forecast_div_value': 0.0, 'inc': 218.25, 'forecast_inc': 0.0, 'name': 'Lena Gold-Mining Public Joint Stock Company Lenzoloto', 'price': 1695.0, '30d_min': 1640.0, '30d_max': 2060.0, '30d_drowdown': -17.71844660194175, '365d_min': 1640.0, '365d_max': 7200.0, '365d_drowdown': -76.45833333333333}
{'symbol': 'LNZL.MOEX', 'ticker': 'LNZL', 'div_value': 15219.5, 'forecast_div_value': 0.0, 'inc': 159.87, 'forecast_inc': 0.0, 'name': 'Lena Gold-Mining Public Joint Stock Company Lenzoloto', 'price': 9520.0, '30d_min': 9590.0, '30d_max': 10680.0, '30d_drowdown': -10.86142322097379, '365d_min': 9590.0, '365d_max': 35000.0, '365d_drowdown': -72.8}
{'symbol': 'NLMK.MOEX', 'ticker': 'NLMK', 'div_value': 41.91, 'forecast_div_value': 40.01, 'inc': 19.64, 'forecast_inc': 18.75, 'name': 'Public Joint Stock Company "Novolipetsk Steel"', 'price': 213.44, '30d_min': 199.8, '30d_max': 223.74, '30d_drowdown': -4.60355770090284, '365d_min': 19

,symbol,ticker,div_value,forecast_div_value,inc,forecast_inc,name,price,30d_min,30d_max,30d_drowdown,365d_min,365d_max,365d_drowdown
0,LNZLP.MOEX,LNZLP,3699.27,0.00,218.25,0.00,Lena Gold-Mining Public Joint Stock Company Le...,1695.00,1640.0,2060.00,-17.72,1640.00,7200.00,-76.46
1,LNZL.MOEX,LNZL,15219.50,0.00,159.87,0.00,Lena Gold-Mining Public Joint Stock Company Le...,9520.00,9590.0,10680.00,-10.86,9590.00,35000.00,-72.80
2,NLMK.MOEX,NLMK,41.91,40.01,19.64,18.75,"Public Joint Stock Company ""Novolipetsk Steel""",213.44,199.8,223.74,-4.60,199.80,279.58,-23.66
3,MTLRP.MOEX,MTLRP,1.17,58.89,0.38,19.10,Mechel PAO,308.25,240.4,305.70,0.83,69.95,329.00,-6.31
4,CHMF.MOEX,CHMF,253.42,310.23,15.99,19.58,Public Joint Stock Company Severstal,1584.80,1452.2,1656.20,-4.31,1262.80,1881.20,-15.76


In [7]:

set_with_dataframe(sheet, new_data_df, include_index=False)